## Bibliotecas

In [2]:
!pip install selenium
!pip install reppy

  Stored in directory: C:\Users\jcss3\AppData\Local\pip\Cache\wheels\ec\5b\61\ed6a2e74060653130204902fce6a93f4da22b55ad36cb3eb60
Successfully built reppy


In [3]:
import pandas as pd
import numpy as np
import scipy as scp
from bs4 import BeautifulSoup
import requests as rq
import re
from requests.exceptions import HTTPError
import urllib
import urllib.request

from selenium import webdriver
from reppy.robots import Robots
from urllib.parse import urlparse
import time
import operator
import nltk

## Crawler

In [4]:
Dominio = ['https://www.capefeargames.com/','https://www.mtgotraders.com/',
           'https://www.wizardscupboard.com/','https://abugames.com/',
           'https://www.cardkingdom.com/','http://www.starcitygames.com/',
           'https://scryfall.com/','http://www.mtgmintcard.com/']
Dados = []
TempoDominio = []

In [5]:
# Função para marcar a page como visitada.
def Visitou (url,paginasVisitadas):
    if url not in paginasVisitadas:
        return False
    else:
        return True

In [114]:
#Função para verificar se a requisição sofreu algum problema.
def verificarRequest(url):
    try:
        response = rq.head(url) 
        if response.status_code == rq.codes.ok:
            return True
        else:
            return False
    except:    
        return False

In [115]:
def CrawlerDinâmico(url):
    #C:\\Users\\Usuário\\Downloads\\chromedriver76\\chromedriver.exe 
    driver = webdriver.Chrome("C:\\Users\\Usuário\\Downloads\\chromedriver76\\chromedriver.exe")
    driver.get(url)
    res = driver.execute_script("return document.documentElement.outerHTML")
    driver.quit
    return res

In [116]:
def verificarRobotTxt(url):
    robots = Robots.fetch(url + '/robots.txt')
    return robots.allowed(url, '*')

In [117]:
# Criar uma Tabela com as informações sobre links.
def Tabela(count,url,site):
    Dados.append(dict(zip(['Número','Link','Site'],[count,url,site])))

## Padrões

In [118]:
def Words():
    goods = ['magic','single','set','catalog','gathering','card','store']
    bad = ['pokemom','yugioh']
    words = [goods,bad]    
    return words

In [119]:
def Padroes():
    
    regex = r'(.+)(magic_singles-)(.+)(\/)(.+)(\/)(\d+)'
    CapeFearGamesMagicCard = re.compile(regex)
    
    regex2 = r'(.)*(store)(.)+'
    MtGoTradersMagicCard = re.compile(regex2)
    
    regex3 = r'(.)*(p-)(\d+)(\.html)'
    WizardCupBoardMagicCard = re.compile(regex3)
    
    regex4 = '(.)*(magic-the-gathering\/)(singles\/)(product-detail\/)(.+)'
    AbuGamesMagicCard = re.compile(regex4)

    regex5 = r'(.+)(mtg\/)(.+)'
    CardKingdomMagicCard = re.compile(regex5)

    regex6 = r'(.+)(magic_the_gathering\/)(product\/)(\d+)(\/\d+)*'
    StarCityMagicCard = re.compile(regex6)

    regex7 = r'(.+)(card)(.+)(\d+)(.+)'
    ScryfallMagicCard = re.compile(regex7)

    regex8 = r'(.+)(mtg\/)(singles\/)(.+\/)(.+)'
    MtgMIntCard = re.compile(regex8)
    
    padroesLinks = [CapeFearGamesMagicCard,CardKingdomMagicCard,AbuGamesMagicCard,ScryfallMagicCard,
                    StarCityMagicCard,WizardCupBoardMagicCard,MtGoTradersMagicCard,MtgMIntCard]
    return padroesLinks

In [124]:
def Score(url,siteInicial,texto):
    score = 0
    
    regexMoney = re.compile(r'(\$)(\d+)(.)(\d{2})')
    
    # Dentro do Dominio +1
    if siteInicial in url:
        score = score + 1
        
    # url contem alguma palavra da lista de palavres que aparecem no link relevante +2   
    for palavra in Words()[0]:
        #casefold ignorar sensitive-case
        if palavra.casefold() in url.casefold():
            score = score + 3
            
    # url fullmatch com link relevante +5
    for padrao in Padroes():
        if padrao.fullmatch(url):
            score = score + 5
    
    #CArdpage tem texto sobre valor da carta. +10
    if regexMoney.match(texto):
        score = score + 10
    
     # url contem alguma palavra da lista de palavras ruins(pokemom, yugioh ...) zera o score.  
    for palavra in Words()[1]:
        #casefold ignorar sensitive-case
        if palavra.casefold() in url.casefold():
            score = 0
    return score

## * Heurística

In [125]:
def CardsCrawler(url):
    print('Dominio: ' + url.geturl())
    count = 0
    score = 0
    paginas = [(url,score)]
    

    paginasVisitadas = []
    robots = Robots.fetch(url.geturl() + '/robots.txt')
    while count < 1000 and len(paginas) > 0:
        paginas.sort(key=operator.itemgetter(1),reverse=True)
        primeiraPagina = paginas.pop(0)
        # Verifica se a pagina foi visitada.Se foi, procuro o proximo link da lista que não foi visitado.
        # Verifico se o hostname de cada pagina é igual ao do dominio, se não, rocuro o proximo link da lista (navegar apenas dentro do site)
        # Verifico se hostname existe , se não , procuro o proximo link que tenha.
        # Verifico se é possivel crawlear a pagina de acordo com as regras do robot.txt , se não , procuro o proximo link.
        # Verifico se a respota da requisição funciona
        while Visitou(primeiraPagina[0].geturl(),paginasVisitadas) == True or primeiraPagina[0].hostname != url.hostname or primeiraPagina[0].hostname == None or robots.allowed(primeiraPagina[0].geturl(), '*') == False or verificarRequest(primeiraPagina[0].geturl()) == False:
            #print(Visitou(primeiraPagina[0].geturl(),paginasVisitadas))
            #print(primeiraPagina[0].hostname != url.hostname)
            #print(primeiraPagina[0].hostname == None)
            #print(robots.allowed(primeiraPagina[0].geturl(), '*'))
            #print(verificarRequest(primeiraPagina[0].geturl()))
            if len(paginas) == 0:
                print('lascou@!')
                return
            primeiraPagina = paginas.pop(0)
    
        # Verficar se o site é dinamico.
        if (Dominio[3] in url.geturl()) or (Dominio[6] in url.geturl()):
            texto = CrawlerDinâmico(primeiraPagina[0].geturl())
        else:
            #request - usando o content type
            codigo_fonte = rq.get(primeiraPagina[0].geturl(),headers={'content-type': 'text/html'})
            # texto do codigo_fonte.
            texto = codigo_fonte.text
        soup = BeautifulSoup(texto,'html.parser')

        # Encontrar todos os links da pagina(url) filtrando pelas tags li ul tr td.
        for lista in soup.findAll(re.compile(r'li|ul|td|tr')):
            for link in lista.findAll('a',href = True):
                
                href = link.get('href')
                # Correção de href relativos para absolutos.
                href = urllib.parse.urljoin(url.geturl(),href)
                
                #TExto ancora
                ancoraTexto = link.text
                
                #Parent text
                parent_text = re.sub('[\s+]', '', link.parent.text)
                if len(parent_text) > 200:
                    parent_text = ''
                
                #concatenar textos da ancora e da tag pai.
                urltexto = ancoraTexto + parent_text

                paginas.append((urlparse(href),Score(href,url.geturl(),urltexto)))
            
        # Marco como visitada a urlparse    
        paginasVisitadas.append(primeiraPagina[0].geturl())
        
        #esperar 500 milisegundos para visitar o prox link.
        time.sleep(.5)
        
        count = count + 1
        print(count,'-',primeiraPagina[0].geturl(),'Score: ',primeiraPagina[1],' Qtd Paginas=', len(paginas))
        
        # Salvar a page localmente
        with open("Paginas/PaginasHeuristica/"+str(url.hostname)+str(count)+".html", 'w' , encoding='utf-8') as outfile:
            outfile.write(texto)
        
        Tabela(count,primeiraPagina[0].geturl(),url.hostname)

In [ ]:
CardsCrawler(urlparse(Dominio[2]))

Dominio: https://www.wizardscupboard.com/
1 - https://www.wizardscupboard.com/ Score:  0  Qtd Paginas= 386
2 - https://www.wizardscupboard.com/card-combos-t-3.html Score:  9  Qtd Paginas= 716
3 - https://www.wizardscupboard.com/dragon-sculptures-ip-11.html Score:  6  Qtd Paginas= 1155
4 - https://www.wizardscupboard.com/company-details-ip-5.html Score:  6  Qtd Paginas= 1502
5 - https://www.wizardscupboard.com/shipping-returns-ip-6.html Score:  6  Qtd Paginas= 1781
6 - https://www.wizardscupboard.com/grading-ip-9.html Score:  6  Qtd Paginas= 2056


From cffi callback <function _verify_callback at 0x00000258386558C8>:
Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\OpenSSL\SSL.py", line 306, in wrapper
    @wraps(callback)
KeyboardInterrupt
From cffi callback <function _verify_callback at 0x0000025838655EA0>:
Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\OpenSSL\SSL.py", line 306, in wrapper
    @wraps(callback)
KeyboardInterrupt
From cffi callback <function _verify_callback at 0x000002583DD33378>:
Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\OpenSSL\SSL.py", line 306, in wrapper
    @wraps(callback)
KeyboardInterrupt


In [123]:
'single' in 'singles_biuebg_ci_112'

True

### Download pages do Dominio.

In [128]:
for url in Dominio:
    inicio = time.time()
    CardsCrawler(urlparse(url))
    fim = time.time()
    tempo = fim - inicio
    print('Tempo do Site: ',tempo, 'Site: ', url)
    TempoDominio.append(dict(zip(['Dominio','Tempo'],[url,tempo])))

Dominio: https://www.capefeargames.com/
1 - https://www.capefeargames.com/ Score:  0  Qtd Paginas= 4546
2 - https://www.capefeargames.com/buylist Score:  1  Qtd Paginas= 8514
3 - https://www.capefeargames.com/faq__policies Score:  1  Qtd Paginas= 13003
4 - https://www.capefeargames.com/specials Score:  1  Qtd Paginas= 17632
5 - https://www.capefeargames.com/products/multi_search Score:  1  Qtd Paginas= 22121
6 - https://www.capefeargames.com/retail_store Score:  3  Qtd Paginas= 26612
7 - https://www.capefeargames.com/catalog/magic_singles/8 Score:  19  Qtd Paginas= 31189
8 - https://www.capefeargames.com/catalog/event_registration-magic_events-prerelease_events/4277 Score:  15  Qtd Paginas= 35732
9 - https://www.capefeargames.com/products/search?q=commander+2019 Score:  1  Qtd Paginas= 40373
10 - https://www.capefeargames.com/catalog/magic_singles-throne_of_eldraine/oko_thief_of_crowns/606112 Score:  29  Qtd Paginas= 44859
11 - https://www.capefeargames.com/catalog/magic_singles-throne

From cffi callback <function _verify_callback at 0x000002734EBBF7B8>:
Traceback (most recent call last):
  File "C:\Users\Usuário\Anaconda3\lib\site-packages\OpenSSL\SSL.py", line 306, in wrapper
    @wraps(callback)
KeyboardInterrupt


SSLError: HTTPSConnectionPool(host='www.capefeargames.com', port=443): Max retries exceeded with url: /catalog/magic_singles-ravnica_allegiance-ravnica_allegiance/4267 (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))

#### * Teste

In [66]:
#request - usando o content type
codigo_fonte = rq.get(Dominio[0],headers={'content-type': 'text/html'})
# texto do codigo_fonte.
texto = codigo_fonte.text
soup = BeautifulSoup(texto,'html.parser')

# lista de tags que podem conter links para listas de cartas.
listasLinks = soup.findAll(re.compile(r'li|ul|td|tr'))

#print(listasLinks)
teste = []
for lista in soup.findAll(re.compile(r'li|ul|td|tr')):
    for link in lista.findAll('a',href = True):
        href = link.get('href')
        #texto
        urltexto = link.text
        
        #parents
        #print(link.parent.text.split('\n')
        parent_text = re.sub('[\s+]', '', link.parent.text)
        if len(parent_text) > 200:
            parent_text = ''
        if href not in teste:
            teste.append((href,urltexto,parent_text))
                         
                         
count = 0
for item in teste:
    count = count + 1
    print(item[2])

Home
SellUsCards
FAQ&Policies
Specials
Multi-Search
RetailStore
Home
SellUsCards
FAQ&Policies
Specials
Multi-Search
RetailStore






Home
SellusCards
FAQ&Policies
Specials
Multi-Search
RetailStore
Home
SellusCards
FAQ&Policies
Specials
Multi-Search
RetailStore






Oko,ThiefofCrowns$39.95
OnceUponaTime$13.95
Garruk,CursedHuntsman$11.95
GildedGoose$6.95
TheCircleofLoyalty$3.95
Oko,ThiefofCrowns$39.95
OnceUponaTime$13.95
Garruk,CursedHuntsman$11.95
GildedGoose$6.95
TheCircleofLoyalty$3.95
ModernHorizonsBoosterBox$219.99
CoreSet2020BoosterBox$99.99
CoreSet2020Bundle$39.95
ModernHorizonsBoosterPack$7.99
CoreSet2020BoosterPack$3.99
ModernHorizonsBoosterBox$219.99
CoreSet2020BoosterBox$99.99
CoreSet2020Bundle$39.95
ModernHorizonsBoosterPack$7.99
CoreSet2020BoosterPack$3.99
TheRiseofQueensdale$64.99
Expancity$59.99
TrekkingtheNationalParks$49.99
Sagrada$44.99
TheRiseofQueensdale$64.99
Expancity$59.99
TrekkingtheNationalParks$49.99
Sagrada$44.99
MonsterRocks:28MagicTheGatheringTokendiceColle

CommanderAnthologyVolumeII
Commander'sArsenal(Foil)
CommanderSingles(Oversized)
ConspiracySetsConspiracyConspiracy:TaketheCrown
Conspiracy
Conspiracy:TaketheCrown
Conspiracy
Conspiracy:TaketheCrown
Conspiracy
Conspiracy:TaketheCrown
DuelDecksDuelDecks:Ajanivs.NicolBolasDuelDecks:AnthologyDuelDecks:Blessedvs.CursedDuelDecks:Divinevs.DemonicDuelDecks:Elspethvs.KioraDuelDecks:Elspethvs.TezzeretDuelDecks:Elvesvs.GoblinsDuelDecks:Elvesvs.InventorsDuelDecks:Garrukvs.LilianaDuelDecks:Heroesvs.MonstersDuelDecks:Izzetvs.GolgariDuelDecks:Jacevs.ChandraDuelDecks:Jacevs.VraskaDuelDecks:JapaneseJacevs.ChandraDuelDecks:Knightsvs.DragonsDuelDecks:Merfolkvs.GoblinsDuelDecks:Mindvs.MightDuelDecks:Nissavs.ObNixilisDuelDecks:Phyrexiavs.theCoalitionDuelDecks:Sorinvs.TibaltDuelDecks:Speedvs.CunningDuelDecks:Venservs.KothDuelDecks:Zendikarvs.Eldrazi
DuelDecks:Ajanivs.NicolBolas
DuelDecks:Anthology
DuelDecks:Blessedvs.Cursed
DuelDecks:Divinevs.Demonic
DuelDecks:Elspethvs.Kiora
DuelDecks:Elspethvs.Tezzeret
Du

DarkLegends
ExclusivePack
PremiumPack1
PremiumPack2
RetroPack1
RetroPack2
DuelistSaga
AnniversaryPack
VideoGamePromos
WC4GBAPromos
2018AdventCalendar
2019Yu-Gi-OhRisingRampageBattlesofLegend:Hero'sRevengeSpeedDuel:AttackfromtheDeepDarkNeostormStructureDeck:OrderoftheSpellcastersDuelPowerSpeedDuel:ArenaofLostSoulsTheInfinityChasersStructureDeck:SoulburnerSavageStrikeLegendaryDuelists:SistersoftheRose2019GoldSarcophagusTinPromosDuelDevastatorLegendaryDuelists:ImmortalDestiny
RisingRampage
BattlesofLegend:Hero'sRevenge
SpeedDuel:AttackfromtheDeep
DarkNeostorm
StructureDeck:OrderoftheSpellcasters
DuelPower
SpeedDuel:ArenaofLostSouls
TheInfinityChasers
StructureDeck:Soulburner
SavageStrike
LegendaryDuelists:SistersoftheRose
2019GoldSarcophagusTinPromos
DuelDevastator
LegendaryDuelists:ImmortalDestiny
RisingRampage
BattlesofLegend:Hero'sRevenge
SpeedDuel:AttackfromtheDeep
DarkNeostorm
StructureDeck:OrderoftheSpellcasters
DuelPower
SpeedDuel:ArenaofLostSouls
TheInfinityChasers
StructureDeck:S